<a href="https://colab.research.google.com/github/lukehinsy/GeospatialPractice/blob/main/mapping_with_yelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Setup





In [1]:
%%capture
#--this just suppresses output

# Install Uber's open-source hexagonal tesselation package
!pip install h3

# Contextilly helps grab basemaps for Folium package we're using
!pip install contextily

# geojson helps parse geographic data
!pip install geojson

# Colab doesn't like matplotlib 3.5.3, so roll back to 3.1.3
# get "cannot import name '_png' from 'matplotlib'" error
!pip install matplotlib==3.1.3

# OSMnx is the key to accessing OpenStreetMaps data. Data on businesses isn't great,
  # but there are ***lots*** of options there. Better for customizing STREET maps, not
  # BUSINESS maps, so out of scope here, but one function added at end as POC
# !pip install osmnx

# YelpAPI helps us grab far better business location data.
!pip install YelpAPI

In [2]:
# Initial Setup
import json
import requests
import pandas as pd
from numpy import linalg
import matplotlib.pyplot as plt
import matplotlib
import shapely

import geopandas as gpd
import folium
import contextily
from folium.plugins import HeatMap
# import osmnx as ox

import h3
from geojson import Feature, Point, FeatureCollection
from shapely.geometry import shape

from google.colab import userdata
from yelpapi import YelpAPI





# "Minimal" Functions

In [3]:
def get_businesses(location, term, api_key):
  """
  Uses YelpAPI to pull up to 1000 businesses, Lat/Lon, Avg Rating, and
    Number of Ratings (plus distance, but we aren't using that).

  """
  headers = {'Authorization': 'Bearer %s' % api_key}
  url = 'https://api.yelp.com/v3/businesses/search'

  data = []
  for offset in range(0, 1000, 50):
      params = {
          'limit': 50,
          'location': location.replace(' ', '+'),
          'term': term.replace(' ', '+'),
          'offset': offset
      }

      response = requests.get(url, headers=headers, params=params)
      if response.status_code == 200:
          data += response.json()['businesses']
      elif response.status_code == 400:
          print('400 Bad Request')
          break

  result_df = pd.DataFrame({'Name': [], 'Lat': [], 'Lon':[], 'Rating':[], 'RatingCount':[], 'Distance':[]})
  listdic = []
  for result in data:
    name = result['name']
    lat = result['coordinates']['latitude']
    lon = result['coordinates']['longitude']
    rating = result['rating']
    ratingcount = result['review_count']
    distance = result['distance']
    listdic=pd.Series([name,lat,lon,rating,ratingcount,distance], index=['Name', 'Lat','Lon', 'Rating', 'RatingCount', 'Distance'])
    result_df=pd.concat([result_df, listdic.to_frame().T], ignore_index=True)

  return result_df


def MapYelps(df):
  #Set figure size, then add map to that
  f = folium.Figure(width=800, height=400)
  m=folium.Map(tiles='CartoDB positron', control=False).add_to(f)

  #Pull top left, bottom right corners and set map bound box.
  sw = [df.Lat.min(), df.Lon.min()]
  ne = [df.Lat.max(), df.Lon.max()]
  m.fit_bounds([sw,ne])

  #Create a layer, add points to it, then add the layer to your map
  Locations = folium.FeatureGroup(name = "Locations")
  for index, row in df.iterrows():
    html = '''
    <b>Name:</b> {name} <br>
    <b>Rating:</b> {rating}
    '''.format(name = row.Name, rating=row.Rating)

    iframe = folium.IFrame(html)
    popup = folium.Popup(iframe,
                        min_width=200,
                        max_width=120)

    Locations.add_child(folium.Marker(location=[row.Lat,row.Lon], popup = popup))
  m.add_child(Locations)

  #Add ability to turn off/on your layers
  folium.LayerControl().add_to(m)

  return(m)



In [ ]:
restaurants=get_businesses('madison, wisconsin', 'restaurants', userdata.get('YelpAPIKey'))

In [ ]:
groceries=get_businesses('madison, wisconsin', 'groceries', userdata.get('YelpAPIKey'))

In [ ]:
fuel=get_businesses('madison, wisconsin', 'gas station', userdata.get('YelpAPIKey'))

In [ ]:
restaurants['Class']='restaurant'
groceries['Class']='grocery'
fuel['Class']='fuel'

full=restaurants.append(groceries).append(fuel)

<ipython-input-8-3689cf39c50c>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full=restaurants.append(groceries).append(fuel)


In [ ]:
display(full)

,Name,Lat,Lon,Rating,RatingCount,Distance,Class
0,Alqueria Farmhouse Kitchen,39.99029,-83.01299,4.5,241,2084.450163,restaurant
1,Third and Hollywood,39.984957,-83.044013,4.5,816,4567.975258,restaurant
2,South Village Grille,39.93954,-82.99077,4.5,288,4844.684143,restaurant
3,Speck Italian Eatery,39.964221,-83.001242,4.5,29,2279.231395,restaurant
4,Tiger + Lily,39.96356,-83.00021,4.0,413,2313.724736,restaurant
...,...,...,...,...,...,...,...
386,Steve's Market and Deli,40.22298,-83.5534,3.5,2,54788.666196,fuel
387,CVS Pharmacy,40.0051,-82.6594,2.0,11,28334.264155,fuel
388,Murphy Oil USA,40.08565,-82.42814,1.5,2,49239.058049,fuel
389,Hocking Hills Camping & Canoe,39.58398,-82.523745,3.0,3,59565.470196,fuel


The search function is very not awesome, so we can't get too specific, but this will work for our sandbox dataset.   

For example, searching for "Mexican Restaurants" gets plenty of results such as IHOP, Dave & Buster's, etc, but that's a data quality issue, not a coding issue.

Let's map these suckers.

#Hex Binning
Getting a bit more advanced, add code to aggregate these locations to standardized geographic regions that can be scored for their contents.


In [4]:

def Hexify(df, resolution=7):
  """
  Hexify will just add a column to the input df that gives the HEX ID for each point in the dataset.
  DF must include columns "Lat" and "Lon".
  """
  hex_ids = df.apply(lambda row: h3.geo_to_h3(row.Lat, row.Lon, resolution), axis = 1)
  df_result = df.assign(hex_id=hex_ids.values)
  return df_result


def hex_df_to_geojson(df_hex, column_name = "value"):
    """
    Produce the GeoJSON for a dataframe, constructing the geometry from the "hex_id" column
    and with a property matching the one in column_name, for example
    """
    list_features = []

    for i,row in df_hex.iterrows():
        try:
            geometry_for_row = { "type" : "Polygon", "coordinates": [h3.h3_to_geo_boundary(h=row["hex_id"],geo_json=True)]}
            feature = Feature(geometry = geometry_for_row , id=row["hex_id"], properties = {column_name : row[column_name]})
            list_features.append(feature)
        except:
            print("An exception occurred for hex " + row["hex_id"])

    feat_collection = FeatureCollection(list_features)
    geojson_result = json.dumps(feat_collection)
    return geojson_result

def get_color(custom_cm, val, vmin, vmax):
    """
    Scales the color gradient to your specific data range
    """
    return matplotlib.colors.to_hex(custom_cm((val-vmin)/(vmax-vmin)))

def choropleth_map(df_aggreg, column_name = "value", border_color = 'black', fill_opacity = 0.7, color_map_name = "Blues", initial_map = None, zoom=7):
  """
  This is a somewhat complicated route to creating a choropleth only lightly edited from something found online.
  Below, I use Folium's built-in choropleth capabilities, and I believe it's much simpler to understand.
  """
  #colormap
  min_value = df_aggreg[column_name].min()
  max_value = df_aggreg[column_name].max()
  mean_value = df_aggreg[column_name].mean()
  print(f"Colour column min value {min_value}, max value {max_value}, mean value {mean_value}")
  print(f"Hexagon cell count: {df_aggreg['hex_id'].nunique()}")

  # the name of the layer just needs to be unique, put something silly there for now:
  name_layer = "Choropleth " + str(df_aggreg)

  if initial_map is None:
      initial_map = folium.Map(location= [+39.9698749,  -083.0090858], zoom_start=zoom, tiles="cartodbpositron")

  #create geojson data from dataframe
  geojson_data = hex_df_to_geojson(df_hex = df_aggreg, column_name = column_name)

  # color_map_name 'Blues' for now, many more at https://matplotlib.org/stable/tutorials/colors/colormaps.html to choose from!
  custom_cm = matplotlib.cm.get_cmap(color_map_name)

  folium.GeoJson(
      geojson_data,
      style_function=lambda feature: {
          'fillColor': get_color(custom_cm, feature['properties'][column_name], vmin=min_value, vmax=max_value),
          'color': border_color,
          'weight': 1,
          'fillOpacity': fill_opacity
      },
      name = name_layer
  ).add_to(initial_map)

  return initial_map

# Download and open a dataset of all counties in United States.

In [5]:
from urllib.request import urlopen
import zipfile
from io import BytesIO

myzip = zipfile.ZipFile(BytesIO(urlopen("https://www2.census.gov/geo/tiger/TIGER2022/COUNTY/tl_2022_us_county.zip").read()))

myzip.extractall()

usa = gpd.read_file(r'/content/tl_2022_us_county.shp')


In [6]:
def HexScore(df, ClassWeights, ScoreWt, resolution = 7):
  """
  Results are 2 columns: Hexagon ID and the score for that hexagon,
  a weighted average of "ScoreWt", weighted by ClassWeights

  df is your geospatial dataframe, including a column called "Class".
  ClassWeights should be a dict with weights for each Class.
  ScoreWt is a column name representing a row-wise weight,
  such as an evaluative rating, with higher numbers = better.

  """
  hexDF = Hexify(df, resolution)
  Classes = df.Class.unique()
  hexDF['row_hex_score'] = hexDF[ScoreWt] * hexDF.Class.map(ClassWeights)

  df_aggreg=hexDF.groupby(['hex_id']).agg(
      HexScore = pd.NamedAgg( column = 'row_hex_score', aggfunc="mean"),
  )

  return df_aggreg



In [ ]:
HexScore(full.sample(n=1000, random_state=215),resolution=7, ClassWeights = {'grocery':10, 'restaurant':5, 'fuel':2}, ScoreWt = 'Rating').reset_index()

,hex_id,HexScore
0,872a82483ffffff,50.0
1,872a82490ffffff,30.0
2,872a82491ffffff,30.0
3,872a82493ffffff,35.0
4,872a824c4ffffff,50.0
...,...,...
265,872a958c3ffffff,30.0
266,872a95c49ffffff,23.5
267,872a95c4bffffff,10.0
268,872a95cd5ffffff,45.0


In [7]:

def poly_geojson(poly):
  poly_geojson = gpd.GeoSeries(poly).__geo_interface__
  poly_geojson = poly_geojson['features'][0]['geometry']
  return poly_geojson


def MapYelps_allinone(df, markers = False, HexHeat = 'Hex', resolution = 8, zoom = 9, fillGeom=False):
  """
  df should be data frame containing at least columns named Lat and Lon. For Markers, want "Name" and "Rating" too.
  HexHeat should be set to 'Hex' or 'Heat'. Any other value will not return a layer.
  res determines size of hexagons. 8 is a good starting point for county-level work.
  zoom is the starting zoom level *if* you are not using markers = True. If using markers = True, then it will use a boundary box based on marker locations.
  fillGeom: When HexHeat=Hex, this determines whether you fill an outer polygon with ALL polygons. Use 5-digit state+county FIPS code or 2-digit state code.
  """
  f = folium.Figure(width=800, height=400)

  if (markers==True):
    m=folium.Map(tiles='CartoDB positron', control=False).add_to(f)
    sw = [df.Lat.min(), df.Lon.min()]
    ne = [df.Lat.max(), df.Lon.max()]
    m.fit_bounds([sw,ne])
    Locations = folium.FeatureGroup(name = "Locations")

    for index, row in df.iterrows():
      html = '''
      <b>Name:</b> {name} <br>
      <b>Rating:</b> {rating}
      '''.format(name = row.Name, rating=row.Rating)

      iframe = folium.IFrame(html)
      popup = folium.Popup(iframe,
                          min_width=200,
                          max_width=120)
      Locations.add_child(folium.Marker(location=[row.Lat,row.Lon], popup = popup))
    m.add_child(Locations)


  if (markers!=True):
    m=folium.Map(tiles='CartoDB positron', control=False, location = [df.Lat.mean(),df.Lon.mean()], zoom_start=zoom).add_to(f)
  if (HexHeat == 'Heat'):
    points=df[['Lat','Lon']].values.tolist()
    HeatMap(points, name="Heatmap").add_to(m)

  if (HexHeat == 'Hex'):
    # hexed = Hexify(df, resolution = res)
    # df_aggreg=hexed.groupby(['hex_id']).size().reset_index(name='HexScore')
    df_aggreg = HexScore(df, resolution=resolution, ClassWeights = {'grocery':10, 'restaurant':5, 'fuel':2}, ScoreWt = 'Rating').reset_index()

    if (fillGeom==False):
      choropleth_map(df_aggreg, 'HexScore', zoom=zoom, initial_map=m)
    if (fillGeom!=False):
      if fillGeom==True:
        fillGeom='39049'
      if len(fillGeom)==5:
        fillpoly=poly_geojson(usa.geometry[usa.GEOID==fillGeom])
      elif len(fillGeom)==2:
        fillpoly=poly_geojson(usa.geometry[usa.STATEFP==fillGeom].unary_union)
      fillhexes=h3.polyfill_geojson(fillpoly,resolution)
      h3_df = pd.DataFrame([],columns=['h3_id','h3_geo_boundary'])
      for h3_hex in fillhexes:
        h3_geo_boundary = shapely.geometry.Polygon(
            h3.h3_to_geo_boundary(h3_hex,geo_json=True)
        )
        h3_df.loc[len(h3_df)]=[
                      h3_hex,
                      h3_geo_boundary
                  ]
      geoms = [shape(i) for i in h3_df.h3_geo_boundary]
      fillgpd = gpd.GeoDataFrame({'hex_id':h3_df.h3_id,'geometry':geoms})
      fillgpd.crs='EPSG:4269'

      folium.Choropleth(
          geo_data=fillgpd,
          name="choropleth",
          data=df_aggreg,
          columns=["hex_id", "HexScore"],
          key_on="feature.properties.hex_id",
          fill_color="Blues",
          fill_opacity=0.7,
          line_opacity=0.02,
          legend_name="HexScore",
          nan_fill_opacity = .05,
      ).add_to(m)


  folium.LayerControl().add_to(m)
  return(m)

In [ ]:
MapYelps_allinone(full, resolution=7, markers=False, fillGeom=True)

In [ ]:
MapYelps_allinone(test, markers = True, HexHeat = False, fillGeom=False)

In [ ]:
MapYelps_allinone(test, markers = False, HexHeat = 'Heat', fillGeom=False)

### Still work to be done: Should have hex pop-ups list the filtered df when clicked, so you can see what it contains.

Set filGeom= True gets you Franklin County, OH's 5-digit FIPS, just for convenience.
Notice that this DROPS any data outside your fillGeom county/state!

In [ ]:
MapYelps_allinone(test, markers = False, HexHeat = 'Hex', fillGeom=True, zoom = 10)


In [9]:
get_businesses('columbus, ohio', 'pizza', userdata.get('YelpAPIKey'))

,Name,Lat,Lon,Rating,RatingCount,Distance
0,Hounddog's Three Degree Pizza,40.016369,-83.012016,3.9,643,1240.91385
1,Fibonacci’s Pizzeria,40.026027,-83.001788,4.6,28,1615.926821
2,Paulie Gee's Short North,39.986741,-83.00563,4.3,321,2828.076443
3,Bazemore Pizza,39.995358,-83.07222,4.6,11,6497.66439
4,East Coast Pizzeria,40.065782,-83.017966,4.5,113,6232.126558
...,...,...,...,...,...,...
995,101 Beer Kitchen,40.119818,-83.092269,4.2,815,14411.631812
996,Donatos,39.991682,-82.841613,2.1,8,13585.490675
997,Subway,39.998068,-83.007417,2.1,8,1673.807276
998,Pj's Pizza,39.931344,-83.146445,0.0,0,15505.054827


Can plug get_businesses() query directly into mapping fxn (excluding hex scoring):

In [10]:
MapYelps_allinone(get_businesses('columbus, ohio', 'pizza', userdata.get('YelpAPIKey')), markers = False, HexHeat = 'Heat', fillGeom=True, zoom = 10)

Can also search for other things if you'd like (note, this is all reliant on yelp's search performance. Just a toy, really)

In [ ]:
MapYelps_allinone(get_businesses('columbus, ohio', 'groceries', userdata.get('YelpAPIKey')), markers = False, HexHeat = 'Hex', fillGeom=True, zoom = 10, res = 7)

If we choose a larger fill geometry, we can find them once again

In [ ]:
MapYelps_allinone(test, markers = False, HexHeat = 'Hex', fillGeom='39', res=6, zoom = 7)  # Use FIPS = 39, and it will merge all Ohio counties into a state polyfill.

# Mapping using Open Source Mapping's OSMnx
### Resources
* Some good sample queries for filtering columns and rows: https://pygis.io/docs/d_access_osm.html
* OpenStreetMaps tag info: https://taginfo.openstreetmap.org/keys
* OSMnx documentation: https://osmnx.readthedocs.io/en/stable/osmnx.html


In [11]:
!pip install osmnx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 2.7 MB/s eta 0:00:00


In [14]:
import osmnx as ox
def PlotLocations(place, tags, heatmap=True, iconURL=''):
  locations = ox.geometries_from_place(place, tags)
  locations=locations.loc[locations.geometry.geometry.type=='Point']['geometry']
  points=[[point.xy[1][0], point.xy[0][0]] for point in locations.geometry]
  f = folium.Figure(width=1000, height=500)
  m=folium.Map(tiles='CartoDB positron', control=False).add_to(f)
  sw = [locations.loc[(locations.geometry.geometry.type=='Point')].bounds.miny.min(), locations.loc[(locations.geometry.geometry.type=='Point')].bounds.minx.min()]
  ne = [locations.loc[(locations.geometry.geometry.type=='Point')].bounds.maxy.max(), locations.loc[(locations.geometry.geometry.type=='Point')].bounds.maxx.max()]

  m.fit_bounds([sw, ne])

  if heatmap==True:
    HeatMap(points, name="Heatmap").add_to(m)
  Locations = folium.FeatureGroup(name = "Locations")
  for coord in points:
    if iconURL != '':
      Locations.add_child(folium.Marker(location = [coord[0], coord[1]], icon=folium.features.CustomIcon(iconURL,icon_size=(30,30))))
    else:
      folium.Marker(location = [coord[0], coord[1]]).add_to(m)
  m.add_child(Locations)
  folium.LayerControl().add_to(m)
  return(m)


In [15]:
PlotLocations('new york state, USA', {'brand':'Whole Foods Market'}, heatmap = False, 'https://upload.wikimedia.org/wikipedia/commons/thumb/a/a2/Whole_Foods_Market_201x_logo.svg/900px-Whole_Foods_Market_201x_logo.svg.png?20200709125956')

<ipython-input-14-e7c3b0cc63b3>:3: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  locations = ox.geometries_from_place(place, tags)
/usr/local/lib/python3.10/dist-packages/osmnx/_overpass.py:245: UserWarning: This area is 87 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)
